In [7]:
# Initial imports
import warnings
warnings.filterwarnings('ignore')
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json
# Initialize the Panel Extensions (for Plotly)
import panel as pn
import param
pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline
#!pip install yfinance
import yfinance as yf
#!pip install mplfinance
import mplfinance as mpf
import datetime as dt
pn.extension()
# Load .env enviroment variables
load_dotenv()


True

In [229]:
# Define Parameters

range_of_cryptos = ["BTC-USD", "ETH-USD", "BNB-USD", "ADA-USD", "DOGE-USD", "XRP-USD", "LINK-USD" , "THETA-USD", "LTC-USD", "XLM-USD"]
range_of_stocks = ["AMZN", "TSLA", "NEO", "AAPL", "NVDA", "BABA", "NFLX", "DIS", "NKE"]

# Define Functions

#CALCULATE 1 YEARS BACK DATE
def calc_1years_backdate():
    end = dt.datetime.now().date()
    start = end - dt.timedelta(days=365)
    return start
#CALCULATE 2 YEARS BACK DATE
def calc_2years_backdate():
    end = dt.datetime.now().date()
    start = end - dt.timedelta(days=730)
    return start
#CALCULATE 3 YEARS BACK DATE
def calc_3years_backdate():
    end = dt.datetime.now().date()
    start = end - dt.timedelta(days=1095)
    return start


#download crypto data and stock data for 3 years
def download_cryptos_data():
    df = yf.download(range_of_cryptos, period = "3y", group_by = 'ticker', threads = True, progress=False)
    df = df.dropna().copy()
    return df

def download_stocks_data():
    df = yf.download(range_of_stocks, period = "3y", group_by = 'ticker', threads = True, progress=False)
    df = df.dropna().copy()
    return df




def cumulative_return_plot(df):
    cumulative_returns = (1 + df['Close'].pct_change()).cumprod()
    plot = cumulative_returns.hvplot.line(x="Date", rot=45, shared_axes=False).opts(yformatter="%.0f", width=500, xlabel="Time", ylabel="Value", title="Cumulative Return")
    return plot


def MA_plot(df):
    df["30MA"]=df[["Close"]].rolling(30).mean()
    df["60MA"]=df[["Close"]].rolling(60).mean()
    df["90MA"]=df[["Close"]].rolling(90).mean()
    df.dropna(inplace=True)
    plot = df[["Close", "30MA", "60MA", "90MA"]].hvplot.line(x="Date", rot=45, shared_axes=False).opts(yformatter="%.0f", width=500, xlabel="Date", ylabel="Price", title="Moving Average")
    
    
    return plot




In [260]:
# Create a Title for the Dashboard

# Fetch initial data:
crypto_df = download_cryptos_data()
stock_df = download_stocks_data()
fetch_data_button = pn.widgets.Button(name='FETCH HISTORICAL DATA FROM YAHOO FINANCE!', button_type='success')
def update_data(event):
    crypto_df = download_cryptos_data()
    stock_df = download_stocks_data()
fetch_data_button.on_click(update_data)



# Define title and welcome text
title = "#Investment Projection Dasboard"
welcome = "This Dashboard provice a visual analysis of historial value for certain stock and cryptos"


#Create Widget Elements:
crypto_ticker = pn.widgets.Select(name='Crypto Tickes', options=range_of_cryptos)
crypto_year_slider = pn.widgets.IntSlider(name='Years of Crypto Data', start=1, end=3, step=1, value=1)
stock_ticker = pn.widgets.Select(name='Stock Tickers', options=range_of_stocks) 
stock_year_slider = pn.widgets.IntSlider(name='Years of Stock Data', start=1, end=3, step=1, value=1)

# Input for Historical data
#Crypto historical Data:
crypto_input = pn.Column(crypto_ticker, crypto_year_slider, align='center')
@pn.depends(crypto_ticker, crypto_year_slider)
def reactive_crypto_MAchart(crypto_ticker, crypto_year_slider):
    if crypto_year_slider == 1:
        return MA_plot(crypto_df[crypto_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif crypto_year_slider == 2:
        return MA_plot(crypto_df[crypto_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return MA_plot(crypto_df[crypto_ticker])
    
@pn.depends(crypto_ticker, crypto_year_slider)
def reactive_crypto_cumulative(crypto_ticker, crypto_year_slider):
    if crypto_year_slider == 1:
        return cumulative_return_plot(crypto_df[crypto_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif crypto_year_slider == 2:
        return cumulative_return_plot(crypto_df[crypto_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return cumulative_return_plot(crypto_df[crypto_ticker])
crypto_column = pn.Column(crypto_input, reactive_crypto_MAchart, reactive_crypto_cumulative)


#Stock historical Data:
stock_input = pn.Column(stock_ticker, stock_year_slider, align='center')
@pn.depends(stock_ticker, stock_year_slider)
def reactive_stock_MAchart(stock_ticker, stock_year_slider):
    if stock_year_slider == 1:
        return MA_plot(stock_df[stock_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif stock_year_slider == 2:
        return MA_plot(stock_df[stock_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return MA_plot(stock_df[stock_ticker])

@pn.depends(stock_ticker, stock_year_slider)
def reactive_stock_cumulative(stock_ticker, stock_year_slider):
    if stock_year_slider == 1:
        return cumulative_return_plot(stock_df[stock_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif stock_year_slider == 2:
        return cumulative_return_plot(stock_df[stock_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return cumulative_return_plot(stock_df[stock_ticker])
stock_column = pn.Column(stock_input, reactive_stock_MAchart, reactive_stock_cumulative)

historical_data_row = pn.Row(crypto_column, stock_column, width=1100)


#Input for your proforlio
crypto_investment_ticker = pn.widgets.Select(name='Crypto Tickes', options=range_of_cryptos)
crypto_investment = pn.widgets.FloatInput(name='Your Crypto Investment in USD', value=0)
crypto_future_slider = pn.widgets.IntSlider(name='Years of MC projection', start=2, end=5, step=3, value=2)

stock_investment_ticker = pn.widgets.Select(name='Crypto Tickes', options=range_of_cryptos)
stock_investment = pn.widgets.FloatInput(name='Your Stock Investment in USD', value=0)
stock_future_slider = pn.widgets.IntSlider(name='Years of MC projection', start=2, end=5, step=3, value=2)

cal_crypto_MC_button = pn.widgets.Button(name='Calculate MC', button_type='danger')
#@pn.depends(crypto_investment, crypto_future_slider)
#def calc_crypto_MC_data(event):
    #
    #CODING LINE HERE!
    #
    #
#cal_crypto_MC_button.on_click(calc_crypto_MC_data)
cal_stock_MC_button = pn.widgets.Button(name='Calculate MC', button_type='danger')
#@pn.depends(stock_investment, stock_future_slider)
#def calc_stock_MC_data(event):
    #
    #CODING LINE HERE!
    #
    #
#cal_stock_MC_button.on_click(calc_stock_MC_data)

crypto_user_input = pn.Column(crypto_investment_ticker, crypto_investment, crypto_future_slider, cal_crypto_MC_button, align='center')
stock_user_input = pn.Column(stock_investment_ticker, stock_investment, stock_future_slider, cal_stock_MC_button, align='center')
user_input = pn.Row(crypto_user_input, stock_user_input)






In [261]:
# Create a layout for the dashboard
dashboard = pn.WidgetBox(pn.Column(title, fetch_data_button,
                        pn.WidgetBox(pn.Tabs(    
                            ("Historical Data", historical_data_row),
                            ("Your Porforlio Alnalysis", user_input),
                            )), align="center"))

dashboard.servable()


WidgetBox
    [0] Column(align='center')
        [0] Markdown(str)
        [1] Button(button_type='success', name='FETCH HISTORICAL D...)
        [2] WidgetBox
            [0] Tabs
                [0] Row(width=1100)
                    [0] Column
                        [0] Column(align='center')
                            [0] Select(name='Crypto Tickes', options=['BTC-USD', 'ETH-USD', ...], value='BTC-USD')
                            [1] IntSlider(end=3, name='Years of Crypto Data', start=1, value=1)
                        [1] ParamFunction(function)
                        [2] ParamFunction(function)
                    [1] Column
                        [0] Column(align='center')
                            [0] Select(name='Stock Tickers', options=['AMZN', 'TSLA', ...], value='AMZN')
                            [1] IntSlider(end=3, name='Years of Stock Data', start=1, value=1)
                        [1] ParamFunction(function)
                        [2] ParamFunction(function)
                [1] Row
                    [0] Column(align='center')
                        [0] Select(name='Crypto Tickes', options=['BTC-USD', 'ETH-USD', ...], value='BTC-USD')
                        [1] FloatInput(name='Your Crypto I...)
                        [2] IntSlider(end=5, name='Years of MC projection', start=2, step=3, value=2)
                        [3] Button(button_type='danger', name='Calculate MC')
                    [1] Column(align='center')
                        [0] Select(name='Crypto Tickes', options=['BTC-USD', 'ETH-USD', ...], value='BTC-USD')
                        [1] FloatInput(name='Your Stock Investment i...)
                        [2] IntSlider(end=5, name='Years of MC projection', start=2, step=3, value=2)
                        [3] Button(button_type='danger', name='Calculate MC')

In [99]:
crypto_df = download_cryptos_data('a')
crypto_df['ADA-USD'].loc[calc_1years_backdate():dt.datetime.now().date()]

#print(calc_1years_backdate())
#candle_volume_plot(crypto_df['ADA-USD'])

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-07-07,0.104957,0.118803,0.103143,0.117554,0.117554,575537074
2020-07-08,0.117490,0.136554,0.115540,0.129908,0.129908,1161107598
2020-07-09,0.129908,0.129908,0.118261,0.124293,0.124293,659099389
2020-07-10,0.124230,0.124417,0.111033,0.118316,0.118316,513272164
2020-07-11,0.118349,0.126715,0.117927,0.125214,0.125214,393237279
...,...,...,...,...,...,...
2021-07-02,1.332942,1.394397,1.286607,1.394397,1.394397,2159409781
2021-07-03,1.394152,1.441714,1.359664,1.406836,1.406836,2028094099
2021-07-04,1.404008,1.493717,1.382153,1.458184,1.458184,1806362471
